# Analyze city property inventory

In [1]:
from sqlalchemy import create_engine
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [2]:
DB = create_engine('postgresql://jon@localhost:5432/propertydb')
AUG1 = pd.to_datetime('2023-08-01')

In [3]:
def quick_stats(df):
    df = df.groupby(df.type).aggregate({ 'count': 'sum', 'lotarea': 'sum' }).sort_values(by='count')

    # Convert lot area to acres
    df['lotarea'] = df.lotarea / 43560

    # Percent of city neighborhood acreage per parcel type
    df['pct_city'] = 100 * (df.lotarea / 33161.843)

    # Multiple of Schenley Park (somewhat recognizable comparison)
    df['mult_schenley'] = df.lotarea / 456

    return df

### City statistics by `parceltype`

In [4]:
propertiescitypt = pd.read_sql_query("""SELECT parceltype AS type, COUNT(DISTINCT parceleproppgh.parcelid), SUM(lotarea::int) AS lotarea
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
GROUP BY type
ORDER BY count DESC;""", DB)

# Merge permanent city ownership parcel types
propertiescitypt.loc[propertiescitypt.type.isin([
    'Greenway', 'Park', 'Infrastructure Protection', 'City Facility', 'Other Public Use'
]), 'type'] = 'PCO'

quick_stats(propertiescitypt)

,count,lotarea,pct_city,mult_schenley
type,,,,
PLB Transfer,3,0.225367,0.000680,0.000494
Unknown Public Use,10,0.529982,0.001598,0.001162
Unknown,180,83.621671,0.252162,0.183381
CDC Property Reserve,192,11.389555,0.034345,0.024977
PCO,2240,2287.426102,6.897765,5.016285
Public Sale,2712,225.179522,0.679032,0.493815
URA Transfer,2815,200.797153,0.605507,0.440345
Hold For Study,4991,2748.636042,8.288550,6.027711


### URA statistics by `parcelstatus`

In [5]:
propertiesuraps = pd.read_sql_query("""SELECT parcelstatus AS type, COUNT(DISTINCT parcelepropura.parcelid), SUM(lotarea::int) AS lotarea
FROM parcelepropura
JOIN assessments
ON assessments.parcelid = parcelepropura.parcelid
GROUP BY parcelstatus
ORDER BY count DESC;""", DB)

quick_stats(propertiesuraps)

,count,lotarea,pct_city,mult_schenley
type,,,,
Dedicated,1,3.760996,0.011341,0.008248
New,1,0.893802,0.002695,0.001960
Sold,1,0.083563,0.000252,0.000183
Hold-PHDC Transfer Pending,3,0.172957,0.000522,0.000379
Sold - Redevelopment in Progress,3,0.201630,0.000608,0.000442
Hold-PLB Transfer Pending,13,0.832461,0.002510,0.001826
Easement Area,23,11.591942,0.034956,0.025421
RFP/RFQ Issued,33,3.607163,0.010877,0.007910
Hold from Market,57,3.211341,0.009684,0.007042


### Joint city-URA statistics for `Hold for Study`

In [6]:
quick_stats(pd.concat([
    propertiescitypt[propertiescitypt.type == 'Hold For Study'],
    propertiesuraps[propertiesuraps.type == 'Hold For Study']
]))

,count,lotarea,pct_city,mult_schenley
type,,,,
Hold For Study,5601,2902.318802,8.751983,6.364734


### Joint city-URA sale date analysis for `Hold For Study`

In [7]:
propertydates = pd.read_sql_query("""SELECT saledate
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'Hold For Study'
AND saledate != ''
UNION ALL
SELECT saledate
FROM parcelepropura
JOIN assessments
ON assessments.parcelid = parcelepropura.parcelid
WHERE parcelstatus = 'Hold For Study'
AND saledate != '';""", DB)

propertydates['ownershiplength'] = (AUG1 - pd.to_datetime(propertydates.saledate)).dt.days
propertydates['ownershiplength'] = (propertydates.ownershiplength / 365).round(1)

In [8]:
ownershiplength = propertydates.ownershiplength.describe().reset_index()
ownershiplength = ownershiplength.rename(columns={ 'ownershiplength': 'yearcount' })
ownershiplength['realyear'] = AUG1.year - ownershiplength.yearcount

ownershiplength

,index,yearcount,realyear
0,count,5579.000000,-3556.000000
1,mean,52.463434,1970.536566
2,std,27.573959,1995.426041
3,min,0.200000,2022.800000
4,25%,32.600000,1990.400000
5,50%,55.200000,1967.800000
6,75%,73.300000,1949.700000
7,max,150.400000,1872.600000


### Sale date analysis for `URA Transfer`

In [9]:
propertydates = pd.read_sql_query("""SELECT saledate
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'URA Transfer'
AND saledate != '';""", DB)

propertydates['ownershiplength'] = (AUG1 - pd.to_datetime(propertydates.saledate)).dt.days
propertydates['ownershiplength'] = (propertydates.ownershiplength / 365).round(1)

In [10]:
ownershiplength = propertydates.ownershiplength.describe().reset_index()
ownershiplength = ownershiplength.rename(columns={ 'ownershiplength': 'yearcount' })
ownershiplength['realyear'] = AUG1.year - ownershiplength.yearcount

ownershiplength

,index,yearcount,realyear
0,count,2815.000000,-792.000000
1,mean,14.645115,2008.354885
2,std,14.964523,2008.035477
3,min,0.200000,2022.800000
4,25%,6.800000,2016.200000
5,50%,10.700000,2012.300000
6,75%,13.300000,2009.700000
7,max,123.100000,1899.900000


### City facilities and `Hold For Study`

In [11]:
facilities = pd.read_sql_query("""SELECT parcelfacilitiespgh.name, parceltype, parcelstatus
FROM parcelfacilitiespgh
JOIN parcelcentroids
ON parcelcentroids.parcelmbl = parcelfacilitiespgh.parcelmbl
LEFT JOIN parceleproppgh
ON parceleproppgh.parcelid = parcelcentroids.parcelid;""", DB)

##### Short list of facilities

In [12]:
facilities[facilities.parceltype == 'Hold For Study'].sort_values(by='name').head(10)

,name,parceltype,parcelstatus
173,Albert Turk Graham Park Shelter,Hold For Study,Hold For Study
163,Allegheny Northside Senior Center and Hazlett ...,Hold For Study,Hold For Study
372,Ammon Recreation Center,Hold For Study,Hold For Study
25,Asphalt Plant Equipment Shelter,Hold For Study,Hold For Study
27,Asphalt Plant Garage 2,Hold For Study,Hold For Study
26,Asphalt Plant Testing Lab Building,Hold For Study,Hold For Study
185,Banksville Concession Stand,Hold For Study,Hold For Study
184,Banksville School Field 3rd Base Dugout,Hold For Study,Hold For Study
183,Banksville School Field Storage Building,Hold For Study,Hold For Study
153,Beechview Senior and Community Center,Hold For Study,Hold For Study


##### Percent of facilities for each parcel type

In [13]:
facilitiesparceltype = facilities.parceltype.value_counts().reset_index()
facilitiesparceltype['pct'] = facilitiesparceltype['count'] / facilitiesparceltype['count'].sum()
facilitiesparceltype

,parceltype,count,pct
0,Park,191,0.498695
1,Hold For Study,184,0.480418
2,City Facility,4,0.010444
3,URA Transfer,2,0.005222
4,Greenway,1,0.002611
5,Public Sale,1,0.002611


### City green space and `Hold For Study`

In [14]:
green = gpd.GeoDataFrame(
    pd.concat([
        gpd.read_file('input/greenways.geojson'),
        gpd.read_file('input/parks.geojson')
    ])
)

In [15]:
points = gpd.read_postgis("""SELECT parcelcentroids.parcelid, parceltype, ST_MakePoint(lon::float, lat::float) AS geom
FROM parceleproppgh
JOIN parcelcentroids
ON parcelcentroids.parcelid = parceleproppgh.parcelid;""", DB, crs='EPSG:4269')
points = points.to_crs('EPSG:4326')

##### Join points to green space

In [16]:
pointsgreen = gpd.sjoin(points, green, predicate='within', how='inner')

In [17]:
pointsgreenparceltype = pointsgreen.parceltype.value_counts().reset_index()
pointsgreenparceltype['pct'] = pointsgreenparceltype['count'] / pointsgreenparceltype['count'].sum()
pointsgreenparceltype

,parceltype,count,pct
0,Greenway,1039,0.421501
1,Park,859,0.348479
2,Hold For Study,536,0.217444
3,URA Transfer,15,0.006085
4,Public Sale,10,0.004057
5,Infrastructure Protection,4,0.001623
6,Unknown,1,0.000406
7,City Facility,1,0.000406


### Joint facilities-green space and `Hold For Study`

In [18]:
pd.concat([
    facilitiesparceltype[facilitiesparceltype.parceltype == 'Hold For Study'],
    pointsgreenparceltype[pointsgreenparceltype.parceltype == 'Hold For Study']
])['count'].sum()

720

### Joint city-URA zoning for `Hold For Study`

In [19]:
zoning = gpd.read_file('input/zoning.geojson')

In [20]:
h4spoints = gpd.read_postgis("""SELECT parcelcentroids.parcelid, parcelboundaries.wkt AS wkt_boundaries, ST_MakePoint(lon::float, lat::float) AS geom
FROM parceleproppgh
JOIN parcelcentroids
ON parcelcentroids.parcelid = parceleproppgh.parcelid
JOIN parcelboundaries
ON parcelboundaries.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'Hold For Study'
UNION ALL
SELECT parcelcentroids.parcelid, parcelboundaries.wkt AS wkt_boundaries, ST_MakePoint(lon::float, lat::float) AS geom
FROM parcelepropura
JOIN parcelcentroids
ON parcelcentroids.parcelid = parcelepropura.parcelid
JOIN parcelboundaries
ON parcelboundaries.parcelid = parcelepropura.parcelid
WHERE parcelstatus = 'Hold For Study'
;""", DB, crs='EPSG:4269')
h4spoints['geom_boundaries'] = h4spoints.wkt_boundaries.apply(wkt.loads)
h4spoints = h4spoints.to_crs('EPSG:4326')

In [21]:
h4spointszoning = gpd.sjoin(h4spoints, zoning, predicate='within', how='inner')

In [22]:
h4spointszoningtype = h4spointszoning.legendtype.value_counts().reset_index()
h4spointszoningtype['pct'] = h4spointszoningtype['count'] / h4spointszoningtype['count'].sum()
h4spointszoningtype

,legendtype,count,pct
0,Hillside,2055,0.366964
1,Parks,1140,0.203571
2,Single-Unit Detached Residential,679,0.121250
3,Multi-Unit Residential,543,0.096964
4,Two-Unit Residential,492,0.087857
5,Single-Unit Attached Residential,263,0.046964
6,Local Neighborhood Commercial,136,0.024286
7,Urban Industrial,78,0.013929
8,Riverfront,55,0.009821
9,Uptown Public Realm,41,0.007321


### MVA, Zoning and `Hold For Study`

In [23]:
mva = gpd.read_file('input/mva21.zip')

In [24]:
morefeasibleh4s = h4spointszoning[~h4spointszoning.legendtype.isin(['Hillside', 'Parks'])]
morefeasibleh4s = morefeasibleh4s.drop(['index_right'], axis=1)

In [25]:
morefeasibleh4smva = gpd.sjoin(morefeasibleh4s, mva, predicate='within', how='inner')

In [26]:
morefeasibleh4smvatype = morefeasibleh4smva.MVA21.value_counts().reset_index()
morefeasibleh4smvatype['pct'] = morefeasibleh4smvatype['count'] / morefeasibleh4smvatype['count'].sum()
morefeasibleh4smvatype

,MVA21,count,pct
0,G,1101,0.246861
1,H,1018,0.228251
2,E,558,0.125112
3,J,512,0.114798
4,D,429,0.096188
5,I,239,0.053587
6,F,227,0.050897
7,NC,203,0.045516
8,B,129,0.028924
9,C,28,0.006278
